# Machine learning với `H2O`

## Giới thiệu

Tương tự như R, `H2O` phát triển module máy học trên `python`. Từ `h2o` version `3.26.0.7`, H2O cho phép upload ngược lại `MOJO object` để có thể sử dụng trong mô hình.

```python
pip install h2o
```

In [5]:
import h2o

In [6]:
h2o.init(nthreads = -1, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.172-b11, mixed mode)
  Starting server from C:\Users\Admin\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Admin\AppData\Local\Temp\tmpzkoh5o7z
  JVM stdout: C:\Users\Admin\AppData\Local\Temp\tmpzkoh5o7z\h2o_Admin_started_from_python.out
  JVM stderr: C:\Users\Admin\AppData\Local\Temp\tmpzkoh5o7z\h2o_Admin_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,Asia/Bangkok
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_Admin_1mfbvt
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [8]:
airlines_data = h2o.import_file("./99_dataset/allyears2k.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [12]:
airlines_data.head()

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,nan,23,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,nan,14,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,nan,29,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,nan,-2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,nan,33,19,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,21,3,728,730,848,849,PS,1451,NA,80,79,nan,-1,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO
1987,10,22,4,728,730,852,849,PS,1451,NA,84,79,nan,3,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,23,5,731,730,902,849,PS,1451,NA,91,79,nan,13,1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,24,6,744,730,908,849,PS,1451,NA,84,79,nan,19,14,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,25,7,729,730,851,849,PS,1451,NA,82,79,nan,2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO


In [13]:
## Create a GBM model, only to later export it as a MOJO
from h2o.estimators import H2OGradientBoostingEstimator
original_model = H2OGradientBoostingEstimator(ntrees = 1)
original_model.train(x = ["Origin", "Dest"], y = "IsDepDelayed", training_frame=airlines_data)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [20]:
original_model.accuracy

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1571539319179_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,1.0,1.0,1050.0,5.0,5.0,5.0,32.0,32.0,32.0




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.24472926827174102
RMSE: 0.49470119089379705
LogLoss: 0.6825726339390792
Mean Per-Class Error: 0.3823387746720379
AUC: 0.674679575109205
pr_auc: 0.6903387192776655
Gini: 0.3493591502184099

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5084441442563068: 


,,NO,YES,Error,Rate
0,NO,5336.0,15551.0,0.7445,(15551.0/20887.0)
1,YES,2186.0,20905.0,0.0947,(2186.0/23091.0)
2,Total,7522.0,36456.0,0.4033,(17737.0/43978.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-10-20 09:44:08,0.086 sec,0.0,0.499372,0.691891,0.50000,0.000000,1.000000,0.474942
1,,2019-10-20 09:44:09,1.320 sec,1.0,0.494701,0.682573,0.67468,0.690339,1.746532,0.403315



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Origin,652.799866,1.000000,0.606876
1,Dest,422.873474,0.647784,0.393124


<bound method H2OBinomialModel.accuracy of >

In [21]:
import pandas as pd

In [28]:
# Convert sang H2o Frame
test_data = pd.DataFrame({'Origin': ['SAN'], 'Dest': ['SFO']})
test_data_h2o = h2o.H2OFrame(test_data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [30]:
# Dự báo
original_model.predict(test_data_h2o)

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,NO,YES
YES,0.475288,0.524712


**Cách download file MOJO**

In [47]:
original_model_filename = original_model.download_mojo(path="../datasets/model/", get_genmodel_jar=True)

In [49]:
original_model_filename

'E:\\OneDrive - VPBank\\02. Python\\01-python-book\\datasets\\model\\GBM_model_python_1571539319179_1.zip'

In [48]:
# Upload mojo back
mojo_model = h2o.upload_mojo(original_model_filename)

generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1571539319179_7


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,1.0,1.0,1050.0,5.0,5.0,5.0,32.0,32.0,32.0




ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.24472926827174102
RMSE: 0.49470119089379705
LogLoss: 0.6825726339390792
Mean Per-Class Error: 0.3823387746720379
AUC: 0.674679575109205
pr_auc: 0.6903387192776655
Gini: 0.3493591502184099

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5084441442563068: 


,,NO,YES,Error,Rate
0,NO,5336.0,15551.0,0.7445,(15551.0/20887.0)
1,YES,2186.0,20905.0,0.0947,(2186.0/23091.0)
2,Total,7522.0,36456.0,0.4033,(17737.0/43978.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.015621,0.563484,1.746532,1.746532,0.917031,0.564021,0.917031,0.564021,0.027283,0.027283,74.653200,74.653200
1,,2,0.036950,0.550921,1.502525,1.605683,0.788913,0.551348,0.843077,0.556706,0.032047,0.059330,50.252468,60.568347
2,,3,0.050525,0.549358,1.464303,1.567698,0.768844,0.549358,0.823132,0.554732,0.019878,0.079208,46.430346,56.769793
3,,4,0.153213,0.547794,1.434318,1.478303,0.753100,0.547794,0.776195,0.550082,0.147287,0.226495,43.431795,47.830286
4,,5,0.275683,0.535261,1.241884,1.373275,0.652061,0.537736,0.721049,0.544597,0.152094,0.378589,24.188360,37.327530
5,,6,0.318796,0.529430,1.094916,1.335631,0.574895,0.530039,0.701284,0.542628,0.047205,0.425794,9.491624,33.563131
6,,7,0.417254,0.527425,1.045084,1.267072,0.548730,0.527425,0.665286,0.539041,0.102897,0.528691,4.508418,26.707168
7,,8,0.639524,0.524712,0.996991,1.173204,0.523478,0.524900,0.616000,0.534126,0.221601,0.750292,-0.300866,17.320376
8,,9,0.765292,0.517017,0.873938,1.124023,0.458868,0.518435,0.590177,0.531547,0.109913,0.860205,-12.606186,12.402269
9,,10,0.828960,0.508444,0.708765,1.092129,0.372143,0.509748,0.573431,0.529873,0.045126,0.905331,-29.123474,9.212888




Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-10-20 09:44:08,0.086 sec,0.0,0.499372,0.691891,0.50000,0.000000,1.000000,0.474942
1,,2019-10-20 09:44:09,1.320 sec,1.0,0.494701,0.682573,0.67468,0.690339,1.746532,0.403315



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Origin,652.799866,1.000000,0.606876
1,Dest,422.873474,0.647784,0.393124


In [38]:
result = mojo_model.predict(airlines_data.head())

generic prediction progress: |████████████████████████████████████████████| 100%


In [50]:
# Convert từ H2oFrame sang DataFrame
type(h2o.as_list(result))

pandas.core.frame.DataFrame

**Lưu ý**:

- Mô hình `MOJO` có thể load ngược lại vào R và sử dụng cho production

## Tài liệu tham khảo 

- http://docs.h2o.ai/h2o-tutorials/latest-stable/